In [1]:
import pandas as pd
import numpy as np
import warnings  

Archivo homicidios, formato xlsx

In [11]:
archivo_excel = 'homicidios.xlsx'


df_hechos = pd.read_excel(archivo_excel, sheet_name='HECHOS', engine='openpyxl')
df_victimas = pd.read_excel(archivo_excel, sheet_name='VICTIMAS', engine='openpyxl')

ETL para el dataframe 'df_hechos', primera hoja del xlsx.


In [12]:
df_hechos.shape

(696, 21)

In [ ]:
df_hechos.isnull().sum()

In [ ]:
df_hechos.info()

Dado que hay datos de la comuna, ademas de latitud y longitud, se decartan las columnas altura y cruce para evitar retirar observaciones ('filas') de la muestra ('dataframe'), Se descarta tambien la columna de participantes debido a que la info se encuentra en (victima y acusado).

In [15]:
df_hechos=df_hechos.drop(['Altura','Cruce','PARTICIPANTES'], axis=1)

Ahora se inspeccionan valores unicos en algunas columnas

In [8]:
df_hechos['VICTIMA'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

In [9]:
df_hechos['ACUSADO'].unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN'], dtype=object)

In [ ]:
df_hechos['Dirección Normalizada'].unique()

In [ ]:
df_hechos['LUGAR_DEL_HECHO'].unique()

Tambien descarta la direccion normalizada, la calle y lugar del hecho, se aborda el analisis por comunas.

In [16]:
df_hechos=df_hechos.drop(['Calle', 'Dirección Normalizada','LUGAR_DEL_HECHO'], axis=1)

In [17]:
df_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'TIPO_DE_CALLE', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y', 'VICTIMA',
       'ACUSADO'],
      dtype='object')

Se verifican los valores unicos en comunas, ya que debe estar entre 1 y 15, que corresponde a las comunas de capital federal.

In [18]:
df_hechos['COMUNA'].unique()

array([ 8,  9,  1, 11, 15,  4,  7, 12,  3, 13, 14, 10,  6,  2,  5,  0],
      dtype=int64)

Hay dos observaciones con comuna 0, se van a descartar.

In [19]:
df_hechos=df_hechos[df_hechos['COMUNA']!=0]

Se crea una columna con el nombre de la comuna usando una base de datos externa.

In [20]:
df_comunas = pd.read_excel('nombres_comunas.xlsx', engine='openpyxl')

In [21]:
hechos=pd.merge(df_hechos,df_comunas, on='COMUNA', how='inner')

Se traslada la posicion de la columna 'NOMBRE_COMUNA'

In [22]:
nombre_comuna=hechos.pop('NOMBRE_COMUNA')

In [23]:
hechos.insert(10,'NOMBRE_COMUNA',nombre_comuna)

Renombre a las variables 'pos x' y 'pos y'.

In [24]:
hechos=hechos.rename(columns={'pos x':'longitud','pos y':'latitud'})

In [25]:
index_sd=hechos[hechos['HH']=='SD'].index
print(index_sd)

Index([516], dtype='int64')


Al valor SD en la columna hora, se le asigna el valor mas probable dentro de la columna que es 16:00:00

In [26]:
hechos.loc[516, 'HH']=16

In [27]:
import datetime as dt

In [28]:
hechos.loc[516, 'HORA']=dt.time(16,0)

se descarta la geocodificacion plana

In [29]:
hechos=hechos.drop(['XY (CABA)'], axis=1)

Se verifica la existencia de filas duplicadas.

In [30]:
duplicados=hechos.duplicated().sum()
print(duplicados)

0


In [32]:
hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,TIPO_DE_CALLE,COMUNA,NOMBRE_COMUNA,longitud,latitud,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AVENIDA,8,"Villa Soldati, Villa Riachuelo y Villa Lugano.",-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,GRAL PAZ,9,"Liniers, Mataderos y Parque Avellaneda.",-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AVENIDA,1,"Retiro, San Nicolás, Puerto Madero, San Telmo,...",-58.39040293,-34.63189362,MOTO,AUTO


Se guarda en formato csv.

In [30]:
hechos.to_csv('Hechos.csv', index='False')

ETL para el dataframe 'df_victimas'

In [33]:
df_victimas.shape

(717, 10)

Inspección de celdas vacías, duplicados y tipo de datos.


In [ ]:
df_victimas.isnull().sum()

In [34]:
df_victimas.duplicated().sum()

0

In [ ]:
df_victimas.info()

Se cambia el tipo de dato de la variable edad a tipo entero, los string no numericos 'SD', se pasan a valor NaN, para remplazarlos despues por el promedio redondeado

In [35]:
# Reemplazar todos los valores no numéricos con NaN
df_victimas['EDAD'] = pd.to_numeric(df_victimas['EDAD'], errors='coerce')


In [36]:
# Reemplazo los valores NaN con el promedio de la columna
promedio = df_victimas['EDAD'].mean()
df_victimas['EDAD'].fillna(promedio, inplace=True)


C:\Users\57319\AppData\Local\Temp\ipykernel_9592\166703752.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_victimas['EDAD'].fillna(promedio, inplace=True)


In [37]:
df_victimas['EDAD'] = df_victimas['EDAD'].astype(int)


Se verifican datos unicos de cada variable

In [39]:
df_victimas['ROL'].unique()

array(['CONDUCTOR', 'PASAJERO_ACOMPAÑANTE', 'PEATON', 'SD', 'CICLISTA'],
      dtype=object)

In [41]:
df_victimas['VICTIMA'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL'], dtype=object)

In [42]:
df_victimas['SEXO'].unique()

array(['MASCULINO', 'FEMENINO', 'SD'], dtype=object)

In [44]:
df_victimas['SEXO'].value_counts()

SEXO
MASCULINO    545
FEMENINO     166
SD             6
Name: count, dtype: int64

Los hombres representan el 75%, por lo que de los 6 datos 'SD' se asignan 4 a MASCULINO y 2 a FEMENINO de manera aleatoria.

In [45]:
indices_sd=df_victimas[df_victimas['SEXO']=='SD'].index.tolist()

In [46]:
#Seleccion aleatoria de 4 incices para MASCULINO
indices_masculino=np.random.choice(indices_sd, 4, replace=False)
#Remocion de los 4 indices seleccionados de la lista original
indices_restantes=list(set(indices_sd)-set(indices_masculino))
#Los indices restantes se asignan a FEMENICO
indices_femenino=np.random.choice(indices_restantes, 2, replace=False)

Se remplaza en la seleccion aleatoria de los indices.

In [47]:
df_victimas.loc[indices_masculino, 'SEXO']='MASCULINO'
df_victimas.loc[indices_femenino, 'SEXO']='FEMENINO'

Conversion a variable dummy

In [48]:
df_victimas['SEXO'] = df_victimas['SEXO'].apply(lambda x: 1 if x == 'MASCULINO' else (0 if x == 'FEMENINO' else None))


In [49]:
df_victimas['ROL'].value_counts()


ROL
CONDUCTOR               330
PEATON                  267
PASAJERO_ACOMPAÑANTE     80
CICLISTA                 29
SD                       11
Name: count, dtype: int64

Encuanto al 'ROL' (1.5%) son 'SD', como es una variable categorica no ordinal, los 'SD' se van a llenar de manera aleatoria con las vategorias correspondientes a 'CONDUCTOR' (46%) y 'PEATON' (37%), se asignan 6 aleatoriamente a 'CONDUCTOR' y 5 a 'PEATON', respetando las proporciones.

In [50]:
indices_ROL=df_victimas[df_victimas['ROL'] =='SD'].index.tolist()

In [51]:
#Seleccion aleatoria de 6 incices para CONDUCTOR
indices_conductor=np.random.choice(indices_ROL, 6, replace=False)
#Remocion de los 6 indices seleccionados de la lista original
indices_restantes_1=list(set(indices_ROL)-set(indices_conductor))
#Los indices restantes se asignan a PEATON
indices_peaton=np.random.choice(indices_restantes_1, 5, replace=False)

In [52]:
df_victimas.loc[indices_conductor, 'ROL']='CONDUCTOR'
df_victimas.loc[indices_peaton, 'ROL']='PEATON'

In [53]:
df_victimas['ROL'].value_counts()

ROL
CONDUCTOR               336
PEATON                  272
PASAJERO_ACOMPAÑANTE     80
CICLISTA                 29
Name: count, dtype: int64

En la variable 'VICTIMA' se eliminan los 'SD' QUE REPRESENTAN EL 1.2%, se asigna aleatoriamente respetando las proporciones los dos valores mas probables de la variable que corresponden a las categorias 'MOTO' y 'PEATON'.

In [54]:
df_victimas['VICTIMA'].value_counts()

VICTIMA
MOTO         303
PEATON       267
AUTO          94
BICICLETA     29
SD             9
CARGAS         7
PASAJEROS      5
MOVIL          3
Name: count, dtype: int64

In [55]:
indices_victima=df_victimas[df_victimas['VICTIMA']=='SD'].index.tolist()

In [56]:
#Seleccion aleatoria de 5 incices para MOTO
indices_moto=np.random.choice(indices_victima, 5, replace=False)
#Remocion de los 6 indices seleccionados de la lista original
indices_restantes_2=list(set(indices_victima)-set(indices_moto))
#Los indices restantes se asignan a PEATON
indices_peaton1=np.random.choice(indices_restantes_2, 4, replace=False)

In [57]:
df_victimas.loc[indices_moto, 'VICTIMA']='MOTO'
df_victimas.loc[indices_peaton1, 'VICTIMA']='PEATON'

In [58]:
df_victimas['VICTIMA'].value_counts()

VICTIMA
MOTO         308
PEATON       271
AUTO          94
BICICLETA     29
CARGAS         7
PASAJEROS      5
MOVIL          3
Name: count, dtype: int64

Ahora se trabaja con la fecha y hora de fallecimiento

In [59]:
conteo_SD = df_victimas[df_victimas['FECHA_FALLECIMIENTO'] == 'SD']['FECHA_FALLECIMIENTO'].count()
print(conteo_SD)

68


Los valores 'SD' representan el 9% de las observaciones (filas) de la muestra (dataframe), por lo que no es conveniente eliminarlos, por lo que el procedimiento a seguir es asignar la fecha promedio.

Se usa la fecha generica 1950-01-01, para despues remplazarla por la fecha promedio de cada año

In [60]:
df_victimas['FECHA_FALLECIMIENTO'] = df_victimas['FECHA_FALLECIMIENTO'].replace('SD', '1950-01-01 00:00:00')

In [61]:
# Convertir la columna a datetime
df_victimas['FECHA_FALLECIMIENTO'] = pd.to_datetime(df_victimas['FECHA_FALLECIMIENTO'])

# Quitar la hora y mantener solo la fecha
df_victimas['FECHA_FALLECIMIENTO'] = df_victimas['FECHA_FALLECIMIENTO'].dt.date


In [62]:
df_filtrado = df_victimas[df_victimas['FECHA_FALLECIMIENTO'] != pd.to_datetime('1950-01-01').date()]


In [63]:
df_filtrado.shape

(649, 10)

In [64]:
# Convertir la fecha a datetime si no lo está
df_filtrado['FECHA_FALLECIMIENTO'] = pd.to_datetime(df_filtrado['FECHA_FALLECIMIENTO'])

# Agrupar por año y calcular la fecha promedio
fecha_promedio = df_filtrado.groupby('AAAA')['FECHA_FALLECIMIENTO'].mean()

# Convertir a string con formato 'YYYY-MM-DD'
fecha_promedio = fecha_promedio.dt.strftime('%Y-%m-%d')

# Convertir el resultado a DataFrame
fecha_promedio = fecha_promedio.reset_index()


C:\Users\57319\AppData\Local\Temp\ipykernel_13136\554363828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['FECHA_FALLECIMIENTO'] = pd.to_datetime(df_filtrado['FECHA_FALLECIMIENTO'])


In [65]:
# Convertir a formato de fecha
fecha_promedio['FECHA_FALLECIMIENTO'] = pd.to_datetime(fecha_promedio['FECHA_FALLECIMIENTO'])


In [66]:
fecha_promedio = fecha_promedio.rename(columns={'FECHA_FALLECIMIENTO': 'MEAN_DATE_YEAR'})

In [67]:
fecha_promedio

,AAAA,MEAN_DATE_YEAR
0,2016,2016-06-19
1,2017,2017-07-03
2,2018,2018-07-06
3,2019,2019-06-21
4,2020,2020-08-07
5,2021,2021-06-20


Ahora en el df original, se remplaza la fecha generica '1950-01-01', en funcion de la columna año, usando el promedio por año.

In [212]:
from datetime import datetime, date

In [168]:
df_victimas.loc[(df_victimas['AAAA'] == 2016) & (df_victimas['FECHA_FALLECIMIENTO'] == date(1950, 1, 1)), 'FECHA_FALLECIMIENTO'] = date(2016, 6, 19)
df_victimas.loc[(df_victimas['AAAA'] == 2017) & (df_victimas['FECHA_FALLECIMIENTO'] == date(1950, 1, 1)), 'FECHA_FALLECIMIENTO'] = date(2017, 7, 3)
df_victimas.loc[(df_victimas['AAAA'] == 2018) & (df_victimas['FECHA_FALLECIMIENTO'] == date(1950, 1, 1)), 'FECHA_FALLECIMIENTO'] = date(2018, 7, 6)
df_victimas.loc[(df_victimas['AAAA'] == 2019) & (df_victimas['FECHA_FALLECIMIENTO'] == date(1950, 1, 1)), 'FECHA_FALLECIMIENTO'] = date(2019, 6, 21)
df_victimas.loc[(df_victimas['AAAA'] == 2020) & (df_victimas['FECHA_FALLECIMIENTO'] == date(1950, 1, 1)), 'FECHA_FALLECIMIENTO'] = date(2020, 8, 7)
df_victimas.loc[(df_victimas['AAAA'] == 2021) & (df_victimas['FECHA_FALLECIMIENTO'] == date(1950, 1, 1)), 'FECHA_FALLECIMIENTO'] = date(2021, 6, 21)

In [70]:
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,1,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,1,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,1,30,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,1,18,2016-06-19
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,1,29,2016-02-01


In [71]:
df_victimas=df_victimas.rename(columns={'ID_hecho':'ID'})

In [ ]:
df_victimas.describe()

In [73]:
df_victimas.to_csv('victimas.csv', index='False')

Se unifican las bases de datos en un solo dataframe

In [190]:
df_victimas_fatales=df_victimas.merge(hechos, on='ID', how='left')

In [ ]:
df_victimas_fatales.head()

In [127]:
def contar_SD(df, valor):
    q_SD={}
    for columna in df.columns:
        q_SD[columna]=(df[columna]== valor).sum()
    return q_SD

In [192]:
contar_SD(df_victimas_fatales, 'SD')

{'ID': 0,
 'FECHA_x': 0,
 'AAAA_x': 0,
 'MM_x': 0,
 'DD_x': 0,
 'ROL': 0,
 'VICTIMA_x': 0,
 'SEXO': 0,
 'EDAD': 0,
 'FECHA_FALLECIMIENTO': 0,
 'N_VICTIMAS': 0,
 'FECHA_y': 0,
 'AAAA_y': 0,
 'MM_y': 0,
 'DD_y': 0,
 'HORA': 0,
 'HH': 0,
 'TIPO_DE_CALLE': 0,
 'COMUNA': 0,
 'NOMBRE_COMUNA': 0,
 'longitud': 0,
 'latitud': 0,
 'VICTIMA_y': 8,
 'ACUSADO': 21}

In [193]:
df_victimas_fatales['ACUSADO'].value_counts()

ACUSADO
AUTO           210
PASAJEROS      178
CARGAS         150
OBJETO FIJO     67
MOTO            58
SD              21
MULTIPLE        17
BICICLETA        7
OTRO             6
TREN             1
Name: count, dtype: int64

In [194]:
index_acusado=df_victimas_fatales[df_victimas_fatales['ACUSADO']=='SD'].index.tolist()

In [195]:
len(index_acusado)

21

los faltantes se van a asignar aleatoriamente a los 3 valores mas probables respetando la proporcion de estos.
se tienen 21 datos en sd, se van a asignar asi (8, 7, 6) => (autos, pasajeros, cargas).

In [196]:
indices_autos=np.random.choice(index_acusado, 8, replace=False)
index_acusado1=list(set(index_acusado)-set(indices_autos))
indices_pasajeros=np.random.choice(index_acusado1, 7, replace=False)
index_acusado2=list(set(index_acusado1)-set(indices_pasajeros))
indices_cargas=np.random.choice(index_acusado2, 6, replace=False)

In [197]:
df_victimas_fatales.loc[indices_autos, 'ACUSADO']='AUTO'
df_victimas_fatales.loc[indices_pasajeros, 'ACUSADO']='PASAJEROS'
df_victimas_fatales.loc[indices_cargas, 'ACUSADO']='CARGAS'

se descartan las filas con valoes nulos en su ubicacion

In [201]:
df_victimas_fatales=df_victimas_fatales.dropna()

In [203]:
df_victimas_fatales.shape

(715, 24)

In [205]:
df_victimas_fatales=df_victimas_fatales.drop(['FECHA_y', 'AAAA_y','MM_y', 'DD_y','HORA', 'FECHA_FALLECIMIENTO'], axis=1)

In [208]:
df_victimas_fatales['COMUNA']=df_victimas_fatales['COMUNA'].astype(int)

Se va a incorporar una columna con grupos etarios

In [218]:
rangos=[0, 12, 18, 35, 50, 65, 100]
etiquetas=['Niño', 'Adolecente', 'Joven Adulto', 'Adulto', 'Adulto Mayor', 'Anciano' ]

In [219]:
df_victimas_fatales['Grupo_Etario'] = pd.cut(df_victimas_fatales['EDAD'], bins=rangos, labels=etiquetas, right=False)

In [223]:
df_victimas_fatales.head()

,ID,FECHA_x,AAAA_x,MM_x,DD_x,ROL,VICTIMA_x,SEXO,EDAD,N_VICTIMAS,HH,TIPO_DE_CALLE,COMUNA,NOMBRE_COMUNA,longitud,latitud,VICTIMA_y,ACUSADO,Grupo_Etario
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,1,19,1.0,4,AVENIDA,8,"Villa Soldati, Villa Riachuelo y Villa Lugano.",-58.47533969,-34.68757022,MOTO,AUTO,Joven Adulto
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,1,70,1.0,1,GRAL PAZ,9,"Liniers, Mataderos y Parque Avellaneda.",-58.50877521,-34.66977709,AUTO,PASAJEROS,Anciano
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,1,30,1.0,7,AVENIDA,1,"Retiro, San Nicolás, Puerto Madero, San Telmo,...",-58.39040293,-34.63189362,MOTO,AUTO,Joven Adulto
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,1,18,1.0,0,AVENIDA,8,"Villa Soldati, Villa Riachuelo y Villa Lugano.",-58.46503904,-34.68092974,MOTO,PASAJEROS,Joven Adulto
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,1,29,1.0,5,AVENIDA,1,"Retiro, San Nicolás, Puerto Madero, San Telmo,...",-58.38718297,-34.62246630,MOTO,PASAJEROS,Joven Adulto


In [224]:
df_victimas_fatales.to_csv('victimas_fatales.csv', index=False)